In [12]:
!pip install protobuf==3.20.1
!pip install -q transformers sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import json
import numpy as np
import os
import pandas as pd
import torch

from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

In [2]:
with open('../data/raw/Conala/conala-corpus/conala-train.json') as f:
    train = json.load(f)

with open('../data/raw/Conala/conala-corpus/conala-test.json') as f:
    test = json.load(f)

In [3]:
model = AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask_finetune")
tokenizer = AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask_finetune",
    skip_special_tokens=False)

/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:921: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [4]:
test[0]

{'intent': 'How can I send a signal from a python program?',
 'rewritten_intent': 'send a signal `signal.SIGUSR1` to the current process',
 'snippet': 'os.kill(os.getpid(), signal.SIGUSR1)',
 'question_id': 15080500}

In [5]:
test[0]['snippet']

'os.kill(os.getpid(), signal.SIGUSR1)'

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
tokenized_text = tokenizer.encode(test[0]['snippet'], return_tensors="pt").to(device)

In [7]:
outs = model.generate(
            tokenized_text,
            min_length=3,
            max_length=100,
            num_beams=2,
            early_stopping=True
           )

In [8]:
dec = [tokenizer.decode(ids) for ids in outs]
print("Predicted Answer: ", dec)

Predicted Answer:  ['<pad> Equivalent of unix SIGUSR1.</s>']


In [9]:
outs

tensor([[    0,    13, 24376,    18, 14368, 18736,  4256,   794,   102,     4,
             1]], device='cuda:0')

In [10]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable


In [11]:
import evaluate
rouge = evaluate.load('rouge')

In [15]:
results = rouge.compute(predictions=dec, references=[test[0]['rewritten_intent']])

INFO:absl:Using default tokenizer.


In [16]:
results

{'rouge1': 0.13333333333333333,
 'rouge2': 0.0,
 'rougeL': 0.13333333333333333,
 'rougeLsum': 0.13333333333333333}

In [14]:
test[0]['snippet']

'os.kill(os.getpid(), signal.SIGUSR1)'